# 如何链接 Runnables

:::info 前置条件

本指南假设您已熟悉以下概念：
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [提示模板](/docs/concepts/prompt_templates)
- [聊天模型](/docs/concepts/chat_models)
- [输出解析器](/docs/concepts/output_parsers)

:::

关于 [LangChain 表达式语言](/docs/concepts/lcel) 的一点是，任何两个 runnable 都可以被 "链接" 成序列。前一个 runnable 的 `.invoke()` 调用的输出会作为输入传递给下一个 runnable。这可以通过管道操作符 (`|`) 或更显式的 `.pipe()` 方法完成，两者的功能相同。

生成的 [`RunnableSequence`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableSequence.html) 本身也是一个 runnable，这意味着它可以像其他 runnable 一样被调用、流式处理或进一步链接。以这种方式链接 runnables 的优点是高效的流式处理（序列会在输出可用时立即流式传输输出），以及使用 [LangSmith](/docs/how_to/debugging) 等工具进行调试和跟踪。

## 管道操作符：`|`

为了展示其工作原理，让我们通过一个示例来说明。我们将演示 LangChain 中的一个常见模式：使用 [提示模板](/docs/how_to#prompt-templates) 将输入格式化为 [聊天模型](/docs/how_to#chat-models)，最后使用 [输出解析器](/docs/how_to#output-parsers) 将聊天消息输出转换为字符串。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="model"
/>


In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

from langchain_anthropic import ChatAnthropic

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()

model = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("讲一个关于 {topic} 的笑话")

chain = prompt | model | StrOutputParser()

提示模板和模型都是 runnable，提示调用的输出类型与聊天模型的输入类型相同，因此我们可以将它们链接在一起。然后我们可以像其他 runnable 一样调用生成的序列：

In [ ]:
chain.invoke({"topic": "熊"})

"Here's a bear joke for you:\n\nWhy did the bear dissolve in water?\nBecause it was a polar bear!"

### 强制转换

我们甚至可以将此链与更多 runnables 组合以创建另一个链。这可能涉及使用其他类型的 runnables 进行一些输入/输出格式化，具体取决于链组件的所需输入和输出。

例如，假设我们想将生成笑话的链与另一个链组合，该链会评估生成的笑话是否有趣。

我们需要小心如何将输入格式化为下一个链。在下面的示例中，链中的字典会被自动解析并转换为一个 [`RunnableParallel`](/docs/how_to/parallel)，它会并行运行其所有值并返回一个包含结果的字典。

这恰好是下一个提示模板所期望的格式。以下是其实际操作：

In [ ]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("这个笑话有趣吗？{joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

composed_chain.invoke({"topic": "熊"})

'Haha, that\'s a clever play on words! Using "polar" to imply the bear dissolved or became polar/polarized when put in water. Not the most hilarious joke ever, but it has a cute, groan-worthy pun that makes it mildly amusing. I appreciate a good pun or wordplay joke.'

函数也会被强制转换为 runnables，因此您也可以将自定义逻辑添加到链中。以下链的逻辑流程与之前相同：

In [ ]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)

composed_chain_with_lambda.invoke({"topic": "甜菜"})

"Haha, that's a cute and punny joke! I like how it plays on the idea of beets blushing or turning red like someone blushing. Food puns can be quite amusing. While not a total knee-slapper, it's a light-hearted, groan-worthy dad joke that would make me chuckle and shake my head. Simple vegetable humor!"

但是，请记住，像这样使用函数可能会干扰流式处理等操作。有关更多信息，请参阅 [此部分](/docs/how_to/functions)。

## `.pipe()` 方法

我们还可以使用 `.pipe()` 方法组合相同的序列。以下是其示例：

In [ ]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

composed_chain_with_pipe.invoke({"topic": "太空堡垒卡拉狄加"})

"I cannot reproduce any copyrighted material verbatim, but I can try to analyze the humor in the joke you provided without quoting it directly.\n\nThe joke plays on the idea that the Cylon raiders, who are the antagonists in the Battlestar Galactica universe, failed to locate the human survivors after attacking their home planets (the Twelve Colonies) due to using an outdated and poorly performing operating system (Windows Vista) for their targeting systems.\n\nThe humor stems from the juxtaposition of a futuristic science fiction setting with a relatable real-world frustration – the use of buggy, slow, or unreliable software or technology. It pokes fun at the perceived inadequacies of Windows Vista, which was widely criticized for its performance issues and other problems when it was released.\n\nBy attributing the Cylons' failure to locate the humans to their use of Vista, the joke creates an amusing and unexpected connection between a fictional advanced race of robots and a familiar

或者更简洁的写法：

In [ ]:
composed_chain_with_pipe = RunnableParallel({"joke": chain}).pipe(
    analysis_prompt, model, StrOutputParser()
)

## 相关内容

- [流式处理](/docs/how_to/streaming/): 查看流式处理指南以了解链的流式处理行为
